In [1]:
import torch
print(torch.__version__)  # Should NOT have "+cpu"
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


2.4.1+cu121
True
NVIDIA GeForce RTX 2080 SUPER


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 1)  # Simple linear layer

    def forward(self, x):
        return self.linear(x)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleModel().to(device)

# Print model details
print("Model is on:", device)


Model is on: cuda


In [3]:
import os
import shutil
import pandas as pd
from PIL import Image
from tqdm import tqdm
import logging
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from timm.models.vision_transformer import vit_base_patch16_224
from sklearn.model_selection import train_test_split


In [4]:
!pip install pandas pillow tqdm futures


In [5]:
import os
import shutil
import pandas as pd
from PIL import Image
from tqdm import tqdm
import logging
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logging.basicConfig(filename='image_processing.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Define dataset paths
DATASET_PATH = r"E:\21F-52 FYP Project\dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
TEST_PATH = os.path.join(DATASET_PATH, "ISIC_2020_Test_JPEG/ISIC_2020_Test_Input")
TRAIN_LABELS_PATH = os.path.join(DATASET_PATH, "ISIC_2020_Training_GroundTruth.csv")
TEST_METADATA_PATH = os.path.join(DATASET_PATH, "ISIC_2020_Test_Metadata.csv")
PROCESSED_PATH = os.path.join(DATASET_PATH, "processedData")

# Create processed data directories
TRAIN_PROCESSED_PATH = os.path.join(PROCESSED_PATH, "trainData")
TEST_PROCESSED_PATH = os.path.join(PROCESSED_PATH, "testData")
MELANOMA_TRAIN = os.path.join(TRAIN_PROCESSED_PATH, "melanoma")
NON_MELANOMA_TRAIN = os.path.join(TRAIN_PROCESSED_PATH, "non_melanoma")
MELANOMA_TEST = os.path.join(TEST_PROCESSED_PATH, "melanoma")
NON_MELANOMA_TEST = os.path.join(TEST_PROCESSED_PATH, "non_melanoma")


# Ensure directories exist
for folder in [MELANOMA_TRAIN, NON_MELANOMA_TRAIN, MELANOMA_TEST, NON_MELANOMA_TEST]:
    os.makedirs(folder, exist_ok=True)

# Load training labels
df_train = pd.read_csv(TRAIN_LABELS_PATH)

def process_image(image_name, image_folder, label, dest_melanoma, dest_non_melanoma):
    """Processes a single image and moves it to the appropriate folder."""
    src_path = os.path.join(image_folder, image_name + ".jpg")
    
    if not os.path.exists(src_path):
        logging.warning(f"Missing image: {image_name}.jpg")
        return
    
    try:
        img = Image.open(src_path).convert("RGB")
        img = img.resize((224, 224))
        dest_folder = dest_melanoma if label == 1 else dest_non_melanoma
        dest_path = os.path.join(dest_folder, image_name + ".jpg")
        img.save(dest_path)
    except Exception as e:
        logging.error(f"Error processing {image_name}.jpg: {e}")


In [6]:
# Process train dataset with threading
with ThreadPoolExecutor() as executor:
    for _, row in tqdm(df_train.iterrows(), total=df_train.shape[0]):
        executor.submit(process_image, row["image_name"], TRAIN_PATH, row["target"], MELANOMA_TRAIN, NON_MELANOMA_TRAIN)

# Load test metadata
df_test = pd.read_csv(TEST_METADATA_PATH)

def process_test_image(image_name, image_folder, dest_non_melanoma):
    """Processes a single test image and moves it into the non_melanoma folder."""
    src_path = os.path.join(image_folder, image_name + ".jpg")
    
    if not os.path.exists(src_path):
        logging.warning(f"Missing test image: {image_name}.jpg")
        return
    
    try:
        img = Image.open(src_path).convert("RGB")
        img = img.resize((224, 224))
        dest_path = os.path.join(dest_non_melanoma, image_name + ".jpg")
        img.save(dest_path)
    except Exception as e:
        logging.error(f"Error processing test image {image_name}.jpg: {e}")

# Process test dataset with threading
with ThreadPoolExecutor() as executor:
    for _, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
        executor.submit(process_test_image, row["image"], TEST_PATH, NON_MELANOMA_TEST)

print("Processing completed successfully!")


100%|██████████| 10982/10982 [00:02<00:00, 4654.70it/s]


Processing completed successfully!


In [7]:
!pip install timm
!pip install --upgrade jupyter ipywidgets


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, WeightedRandomSampler
from timm.models.vision_transformer import vit_base_patch16_224
from collections import Counter
from PIL import Image
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dir = r"E:\21F-52 FYP Project\dataset\processedData\trainData"
test_dir = r"E:\21F-52 FYP Project\dataset\processedData\testData"

melanoma_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

non_melanoma_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

class ClassSpecificAugmentDataset(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            image = Image.open(path).convert("L")
            if self.classes[target] == "melanoma":
                image = melanoma_transform(image)
            else:
                image = non_melanoma_transform(image)
            return image, target
        except Exception as e:
            print(f"Error loading image: {path} — {e}")
            raise

train_dataset = ClassSpecificAugmentDataset(train_dir)
targets = [label for _, label in train_dataset]
class_counts = Counter(targets)
class_weights = [1.0 / class_counts[label] for label in targets]
sample_weights = torch.DoubleTensor(class_weights)
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=0)

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

class ViTModel(nn.Module):
    def __init__(self, num_classes=2):
        super(ViTModel, self).__init__()
        self.vit = vit_base_patch16_224(pretrained=True)
        self.vit.head = nn.Sequential(
            nn.Dropout(0.5),                # Increased Dropout for regularization
            nn.Linear(768, num_classes)
        )
    def forward(self, x):
        return self.vit(x)

model = ViTModel(num_classes=2).to(device)
class_weights = torch.tensor([1.0 / class_counts[i] for i in range(len(class_counts))]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)  # Lower learning rate & added weight decay for regularization
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=10, patience=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    counter = 0
    best_test_acc = 0  # Initialize best test accuracy for early stopping

    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        scheduler.step()
        train_acc = correct / total
        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        test_correct, test_total = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == labels).sum().item()
                test_total += labels.size(0)
        test_acc = test_correct / test_total

        print(f"📊 Epoch [{epoch+1}/{epochs}] - Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

        # Early stopping based on test accuracy
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"⛔ Early stopping at epoch {epoch+1}")
                break

    model.load_state_dict(best_model_wts)

    # ✅ Final Evaluation after training
    model.eval()

    train_correct, train_total = 0, 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            train_total += labels.size(0)
    final_train_acc = train_correct / train_total

    test_correct, test_total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_correct += (predicted == labels).sum().item()
            test_total += labels.size(0)
    final_test_acc = test_correct / test_total

    print(f"\n✅ Final Training Accuracy: {final_train_acc:.4f}")
    print(f"✅ Final Testing Accuracy: {final_test_acc:.4f}")

train_model(model, train_loader, criterion, optimizer, scheduler, epochs=10, patience=5)


c:\Python38\lib\site-packages\timm\models\vision_transformer.py:93: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(


📊 Epoch [1/10] - Train Loss: 0.0165 | Train Acc: 0.9933 | Test Acc: 0.9566
📊 Epoch [2/10] - Train Loss: 0.0064 | Train Acc: 0.9988 | Test Acc: 0.9672
📊 Epoch [3/10] - Train Loss: 0.0063 | Train Acc: 0.9982 | Test Acc: 0.9669
📊 Epoch [4/10] - Train Loss: 0.0070 | Train Acc: 0.9985 | Test Acc: 0.9674
📊 Epoch [5/10] - Train Loss: 0.0051 | Train Acc: 0.9991 | Test Acc: 0.8095
📊 Epoch [6/10] - Train Loss: 0.0052 | Train Acc: 0.9993 | Test Acc: 0.9729
📊 Epoch [7/10] - Train Loss: 0.0035 | Train Acc: 0.9995 | Test Acc: 0.9619
📊 Epoch [8/10] - Train Loss: 0.0030 | Train Acc: 0.9996 | Test Acc: 0.9689
📊 Epoch [9/10] - Train Loss: 0.0061 | Train Acc: 0.9991 | Test Acc: 0.9717
📊 Epoch [10/10] - Train Loss: 0.0044 | Train Acc: 0.9993 | Test Acc: 0.9725

✅ Final Training Accuracy: 0.9993
✅ Final Testing Accuracy: 0.9729


In [9]:
import torch

torch.save(model.state_dict(), "model.pth")  # Save model


In [10]:
import os
from PIL import Image
import torch
from torchvision import transforms

# Define Model Path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Image Transformations (Same as Training)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5], [0.5])
# ])

# 🟨 Use this only for training images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])



In [11]:
# Corrected Model Path
model_path = r"E:\21F-52 FYP Project\FYP_04_SkinCancer.pth"

# Ensure the file exists before loading
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at: {model_path}")

In [12]:
# Load Your Trained Model
import torch
import torch.nn as nn
import torchvision.models as models

from torchvision.models.vision_transformer import vit_b_16

# Define Model Architecture (ViT-Based)
class SkinCancerViTModel(nn.Module):
    def __init__(self, num_classes=2):
        super(SkinCancerViTModel, self).__init__()
        self.model = vit_b_16(weights=None)  # Initialize ViT without pretrained weights
        num_ftrs = self.model.heads.head.in_features
        self.model.heads.head = nn.Linear(num_ftrs, num_classes)  # Adjust for binary classification

    def forward(self, x):
        return self.model(x)


In [13]:
# Initialize Model
model = SkinCancerViTModel().to(device)

model.load_state_dict(torch.load(model_path, map_location=device), strict=False)

C:\Users\Sanaullah\AppData\Local\Temp\ipykernel_6460\1111948775.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location

_IncompatibleKeys(missing_keys=['model.class_token', 'model.conv_proj.weight', 'model.conv_proj.bias', 'model.encoder.pos_embedding', 'model.encoder.layers.encoder_layer_0.ln_1.weight', 'model.encoder.layers.encoder_layer_0.ln_1.bias', 'model.encoder.layers.encoder_layer_0.self_attention.in_proj_weight', 'model.encoder.layers.encoder_layer_0.self_attention.in_proj_bias', 'model.encoder.layers.encoder_layer_0.self_attention.out_proj.weight', 'model.encoder.layers.encoder_layer_0.self_attention.out_proj.bias', 'model.encoder.layers.encoder_layer_0.ln_2.weight', 'model.encoder.layers.encoder_layer_0.ln_2.bias', 'model.encoder.layers.encoder_layer_0.mlp.0.weight', 'model.encoder.layers.encoder_layer_0.mlp.0.bias', 'model.encoder.layers.encoder_layer_0.mlp.3.weight', 'model.encoder.layers.encoder_layer_0.mlp.3.bias', 'model.encoder.layers.encoder_layer_1.ln_1.weight', 'model.encoder.layers.encoder_layer_1.ln_1.bias', 'model.encoder.layers.encoder_layer_1.self_attention.in_proj_weight', 'mod

In [14]:
# Set Model to Evaluation Mode
model.eval()


SkinCancerViTModel(
  (model): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_

In [23]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Match training preprocessing exactly
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),   # Match training
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Path
test_folder_path = r"E:\21F-52 FYP Project\dataset\processedData\testData"
if not os.path.exists(test_folder_path):
    raise FileNotFoundError(f"Test folder not found: {test_folder_path}")

# Predict
def predict_image(image_path, model, transform):
    image = Image.open(image_path).convert("L")  # Force grayscale (then to 3-channels in transform)
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return "Melanoma" if predicted.item() == 1 else "Non-Melanoma"

# Load model
model.eval()
correct, total = 0, 0

for root, _, files in os.walk(test_folder_path):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, filename)
            prediction = predict_image(image_path, model, test_transform)

            # Ground truth
            folder_name = os.path.basename(root).lower()
            if "melanoma" in folder_name:
                true_label = "Melanoma"
            elif "non" in folder_name:
                true_label = "Non-Melanoma"
            else:
                print(f"⚠️ Skipping: Unknown label → {folder_name}")
                continue

            is_correct = prediction == true_label
            correct += int(is_correct)
            total += 1

            print(f"{filename} → Predicted: {prediction}, Actual: {true_label}, {'✅' if is_correct else '❌'}")

# Final result
if total > 0:
    accuracy = 100 * correct / total
    print(f"\n✅ Final Evaluation Accuracy: {accuracy:.2f}% ({correct}/{total})")
else:
    print("⚠️ No test images found.")


ISIC_0000002.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000004.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000013.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000022.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000022_downsampled.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000026.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000026_downsampled.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000029.jpg → Predicted: Non-Melanoma, Actual: Melanoma, ❌
ISIC_0000029_downsampled.jpg → Predicted: Non-Melanoma, Actual: Melanoma, ❌
ISIC_0000030.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000030_downsampled.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000031.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000031_downsampled.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000035.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000035_downsampled.jpg → Predicted: Melanoma, Actual: Melanoma, ✅
ISIC_0000036.jpg → P

In [ ]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Match training preprocessing
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Path to test data
test_folder_path = r"E:\21F-52 FYP Project\dataset\processedData\testData"
if not os.path.exists(test_folder_path):
    raise FileNotFoundError(f"Test folder not found: {test_folder_path}")

# Get class-to-index mapping using ImageFolder
temp_dataset = ImageFolder(test_folder_path)
class_to_idx = temp_dataset.class_to_idx  # E.g., {'melanoma': 0, 'non_melanoma': 1}
idx_to_class = {v: k for k, v in class_to_idx.items()}  # Reverse mapping

print(f"📂 Class mapping: {class_to_idx}")

# Prediction function
def predict_image(image_path, model, transform):
    image = Image.open(image_path).convert("L")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        probs = F.softmax(output, dim=1)
        _, predicted = torch.max(probs, 1)
    return predicted.item(), probs

# Assume model is already loaded and on correct device
model.eval()

correct, total = 0, 0
true_labels = []
pred_labels = []

for root, _, files in os.walk(test_folder_path):
    folder_name = os.path.basename(root)
    if folder_name not in class_to_idx:
        continue  # Skip folders not in class mapping
    true_label = class_to_idx[folder_name]

    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, filename)
            predicted_class, probs = predict_image(image_path, model, test_transform)

            true_labels.append(true_label)
            pred_labels.append(predicted_class)

            is_correct = predicted_class == true_label
            correct += int(is_correct)
            total += 1

            predicted_label_name = idx_to_class[predicted_class]
            true_label_name = idx_to_class[true_label]

            print(f"{filename} → Predicted: {predicted_label_name}, Actual: {true_label_name}, {'✅' if is_correct else '❌'}")

# Metrics
if total > 0:
    accuracy = 100 * correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary', pos_label=class_to_idx['melanoma'])

    print(f"\n✅ Final Evaluation Accuracy: {accuracy:.2f}% ({correct}/{total})")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")
else:
    print("⚠️ No test images found.")



📂 Class mapping: {'melanoma': 0, 'non_melanoma': 1}
ISIC_0000002.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000004.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000013.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000022.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000022_downsampled.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000026.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000026_downsampled.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000029.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000029_downsampled.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000030.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000030_downsampled.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000031.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000031_downsampled.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000035.jpg → Predicted: melanoma, Actual: melanoma, ✅
ISIC_0000035_downsampled.jpg → Predicted: melano